In [1]:
import tensorflow as tf
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def read_files(files, path):
    ans_all = []
    features = []
    for ans, files in files.items():
        for file in files:
            wave, sr = librosa.load(path + file, mono=True)
            ans_all.append(ans)
#             print(file, ans)
            mfcc = librosa.feature.mfcc(wave, sr)
            mfcc = np.pad(mfcc, ((0,0), (0, 100 - len(mfcc[0]))), mode='constant',
                         constant_values=0)
            features.append(np.array(mfcc))
    labels = to_categorical(ans_all, num_classes=10)
#     labels = ans_all
    print(labels, labels.shape)
    return np.array(features), np.array(labels)

In [3]:
def mean_normalize(features):
    std_value = features.std()
    mean_value = features.mean()
    return (features - mean_value) / std_value

In [4]:
class CNNConfig():
    filter_sizes = [2, 3, 4, 5]
    num_filters = 64
    hidden_dim = 64
    learning_rate = 0.01
    num_epochs = 500
    dropout_keep_prob = 0.5
    print_per_batch = 1
    save_tb_per_batch = 10


In [5]:
class ASRCNN(object):
    def __init__(self, config, width, height, num_classes):
        self.config = config
        self.input_x = tf.placeholder(tf.float32, [None, width, height], name='input_x')
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
        self.keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
#         input_x = tf.transpose(self.input_x, [0,2,1])

        pooled_outputs = []
        for i, filter_size in enumerate(self.config.filter_sizes):
            with tf.name_scope("conv-maxpool-%s" %filter_size):
                print("conv-maxpool-%s" % filter_size)
                conv = tf.layers.conv1d(self.input_x, self.config.num_filters, filter_size,
                                       activation=tf.nn.relu)
                pooled = tf.reduce_max(conv, reduction_indices=[1])
                pooled_outputs.append(pooled)

        num_filters_total = self.config.num_filters * len(self.config.filter_sizes)
        pooled_reshape = tf.reshape(tf.concat(pooled_outputs, 1), [-1, num_filters_total])
                                                     
        fc = tf.layers.dense(pooled_reshape, self.config.hidden_dim, name='fc1')
        fc_drop = tf.contrib.layers.dropout(fc, self.keep_prob)
        self.logits = tf.layers.dense(fc_drop, num_classes, name='fc2')
        
        self.y_pred_cls = tf.argmax(tf.nn.softmax(self.logits), 1, name='pred')
    
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.input_y)
        self.loss = tf.reduce_mean(cross_entropy)
        
        self.optim = tf.train.AdamOptimizer(learning_rate=self.config.learning_rate).minimize(self.loss)
        
        correct_pred = tf.equal(tf.argmax(self.input_y, 1), self.y_pred_cls)

        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [6]:
def batch_iter(train_features, train_labels, shuffle=True):
    data_size = len(train_features)
    batch_size = 20
    num_batch_per_epoch = int(data_size/ batch_size)
    
    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffle_features = train_features[shuffle_indices]
        shuffle_labels = train_labels[shuffle_indices]
    else:
        shuffle_features = train_features
        shuffle_labels = train_labels
        
    for batch_num in range(num_batch_per_epoch):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        
        yield (shuffle_features[start_index:end_index], shuffle_labels[start_index:end_index])

In [7]:
def fetch_batch(epoch, batch_index, batch_size, train_features, train_labels):
    n_batches = len(train_labels) // batch_size
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(len(train_labels), size=batch_size)
    x_batch = train_features[indices]
    y_batch = train_labels[indices]
    return x_batch, y_batch

In [8]:
def train(argv=None):
    train_and_val_path = "./numbersRec/recordings/"
    test_path = "./numbersRec/test/"
    train_files, valid_files, test_files = load_files()
    train_features, train_labels = read_files(train_files, train_and_val_path)
    train_features = mean_normalize(train_features)
    print("read train files down")
    valid_features, valid_labels = read_files(valid_files, train_and_val_path)
    valid_features = mean_normalize(valid_features)
    print("read valid files down")
    test_features, test_labels = read_files(test_files, test_path)
    test_features = mean_normalize(test_features)
    print("read test files down")
    
    width = 20 # mfcc feature
    height = 100 # (max)length of utterance
    classes = 10 # digits
    
    config = CNNConfig
    cnn = ASRCNN(config, width, height, classes)
#     session = tf.Session()
#     session.run(tf.global_variables_initializer())
#     saver = tf.train.Saver(tf.global_variables())
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    with tf.Session() as sess:
        init.run()
        checkpoint_path = os.path.join("cnn_model", "model.ckpt")
        tensorboard_train_dir = "tensorboard/train"
        tensorboard_valid_dir = "tensorboard/valid"

        if not os.path.exists(tensorboard_train_dir):
            os.makedirs(tensorboard_train_dir)
        if not os.path.exists(tensorboard_valid_dir):
            os.makedirs(tensorboard_valid_dir)
        tf.summary.scalar("loss", cnn.loss)
        tf.summary.scalar("accuracy", cnn.acc)
        merged_summary = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(tensorboard_train_dir)
        valid_writer = tf.summary.FileWriter(tensorboard_valid_dir)

        total_batch = 0
        batch_size = 100
        n_batches = len(train_labels) // batch_size

        for epoch in range(config.num_epochs):
            print("Epoch:" ,epoch+1)
    #         batch_train = batch_iter(train_features, train_labels)
    #         for x_batch, y_batch in batch_iter(train_features, train_labels):
    #             print(config.dropout_keep_prob)
            for iteration in range(n_batches):
                x_batch, y_batch = fetch_batch(epoch, iteration, batch_size, train_features, train_labels)
                total_batch +=1
    #             feed_dict = feed_data(cnn, x_batch, y_batch, config.dropout_keep_prob)
                feed_dict = {cnn.input_x:x_batch, cnn.input_y:y_batch, cnn.keep_prob:config.dropout_keep_prob}
                sess.run(cnn.optim, feed_dict=feed_dict)
                if total_batch % config.print_per_batch == 0:
                    train_loss, train_accuracy = sess.run([cnn.loss, cnn.acc], feed_dict=feed_dict)
                    valid_loss, valid_accuracy = sess.run([cnn.loss, cnn.acc], feed_dict={cnn.input_x: valid_features,
                        cnn.input_y:valid_labels, cnn.keep_prob:1.0
                    })
                    print("Steps:" + str(total_batch))
                    print("train_loss: " + str(train_loss) + " train accuracy: " 
                          + str(train_accuracy)+ "\tvalid_loss:" + str(valid_loss)
                         + " valid accuracy: " + str(valid_accuracy))
                if total_batch % config.save_tb_per_batch == 0:
                    train_s = sess.run(merged_summary, feed_dict=feed_dict)
                    train_writer.add_summary(train_s, total_batch)
                    valid_s = sess.run(merged_summary, feed_dict={cnn.input_x: valid_features, cnn.input_y: valid_labels,
                                                                     cnn.keep_prob: config.dropout_keep_prob})
                    valid_writer.add_summary(valid_s, total_batch)
            saver.save(sess, checkpoint_path, global_step=epoch)
        test_loss, test_accuracy = sess.run([cnn.loss, cnn.acc],
                                           feed_dict={cnn.input_x: test_features, cnn.input_y: test_labels,
                                                      cnn.keep_prob: 1.0})
        print('test_loss: ' + str(test_loss) + ' test accuracy: ' + str(test_accuracy))



In [9]:
def read_test_wave(path):
    files = os.listdir(path)
    feature = []
    features = []
    label = []
    for wav in files:
        if not wav.endswith(".wav"): continue
        ans = int(wav[0])
        wave, sr = librosa.load(path + wav, mono=True)
        label.append(ans)
        mfcc = librosa.feature.mfcc(wave, sr)
        mfcc = np.pad(mfcc, ((0, 0), (0, 100 - len(mfcc[0]))), mode='constant', constant_values=0)
        feature.append(np.array(mfcc))   
    features = mean_normalize(np.array(feature))
    return features,label


In [10]:
def test(path):
    features, label = raed_test_wave(path)
    print("loading ASRCNN model...")
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph("cnn_model/model.ckpt-999.meta")
        saver.restore(sess, tf.train.latest_chekpoint("cnn_model"))
        graph = tf.get_default_graph()
        input_x = graph.get_tensor_by_name("input_x:0")
        pred = graph.get_tensor_by_name("pred:0")
        keep_prob = graph.get_tensor_by_name("keep_prob:0")
        for i in range(0, len(label)):
            feed_dict = {input_x: features[i].reshape(1, 20, 100), keep_prob: 1.0}
            test_output = sess.run(pred, feed_dict=feed_dict)
            
            print("="*15)
            print("真实lable: %d" % label[i])
            print("识别结果为:"+str(test_output[0]))
        print("Congratulation!")

In [11]:
def load_files():
    path = "./numbersRec/recordings"
    files = os.listdir(path)
    train_files = [] 
    val_files = []
    for i in range(0, 10):
        raw_data = []
        raw_target = []
        for file in files:
            if file[0] == str(i):
                raw_data.append(file)
                raw_target.append(i)
#         print(raw_data)
        train_X, val_X, train_y, val_y = train_test_split(raw_data,
                                           raw_target,
                                           test_size = 0.2,
                                           random_state = 0)
        train_files += train_X
        val_files += val_X
    test_path = "./numbersRec/test"
    test_files = os.listdir(test_path)
    train_dict = {}
    val_dict = {}
    test_dict = {}
    for i in range(0, 10):
        train_dict[i] = []
        val_dict[i] = []
        if i != 2:
            test_dict[i] = []
    for file in train_files:
        train_dict[int(file[0])].append(file)
    for file in val_files:
        val_dict[int(file[0])].append(file)
    for file in test_files:
        test_dict[int(file[0])].append(file)

    return train_dict, val_dict, test_dict

In [12]:
train()

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] (3111, 10)
read train files down
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] (780, 10)
read valid files down
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]] (9, 10)
read test files down
conv-maxpool-2
Instructions for updating:
Use keras.layers.conv1d instead.
Instructions for updating:
Colocations handled automatically by placer.
conv-maxpool-3
conv-maxpool-4
conv-maxpool-5
Instructions for updating:
Use keras.layers.dense instead.

For more information, please see:
  * https:

Steps:57
train_loss: 3.7390018 train accuracy: 0.21	valid_loss:2.6293938 valid accuracy: 0.30769232
Steps:58
train_loss: 2.893011 train accuracy: 0.24	valid_loss:2.4944062 valid accuracy: 0.30128205
Steps:59
train_loss: 3.1953855 train accuracy: 0.26	valid_loss:2.352065 valid accuracy: 0.32948717
Steps:60
train_loss: 2.6795928 train accuracy: 0.28	valid_loss:2.3086495 valid accuracy: 0.30897436
Steps:61
train_loss: 2.608688 train accuracy: 0.23	valid_loss:2.2562158 valid accuracy: 0.28846154
Steps:62
train_loss: 2.4556642 train accuracy: 0.29	valid_loss:2.3761742 valid accuracy: 0.27564102
Epoch: 3
Steps:63
train_loss: 2.9047534 train accuracy: 0.25	valid_loss:2.4049032 valid accuracy: 0.2794872
Steps:64
train_loss: 2.4906712 train accuracy: 0.3	valid_loss:2.1825523 valid accuracy: 0.2974359
Steps:65
train_loss: 2.0696173 train accuracy: 0.34	valid_loss:2.0389702 valid accuracy: 0.32820514
Steps:66
train_loss: 2.4131565 train accuracy: 0.29	valid_loss:1.9037552 valid accuracy: 0.364102

Steps:142
train_loss: 1.842999 train accuracy: 0.35	valid_loss:1.7185187 valid accuracy: 0.4730769
Steps:143
train_loss: 1.9360684 train accuracy: 0.42	valid_loss:1.6759228 valid accuracy: 0.4628205
Steps:144
train_loss: 1.8422415 train accuracy: 0.36	valid_loss:1.628544 valid accuracy: 0.4602564
Steps:145
train_loss: 1.6867577 train accuracy: 0.41	valid_loss:1.6311957 valid accuracy: 0.4653846
Steps:146
train_loss: 1.6696094 train accuracy: 0.44	valid_loss:1.6290523 valid accuracy: 0.4576923
Steps:147
train_loss: 2.0459826 train accuracy: 0.41	valid_loss:1.6092209 valid accuracy: 0.45641026
Steps:148
train_loss: 1.576368 train accuracy: 0.51	valid_loss:1.6010336 valid accuracy: 0.4679487
Steps:149
train_loss: 1.8414088 train accuracy: 0.42	valid_loss:1.572058 valid accuracy: 0.47948718
Steps:150
train_loss: 1.6438396 train accuracy: 0.51	valid_loss:1.5453708 valid accuracy: 0.4948718
Steps:151
train_loss: 1.6045811 train accuracy: 0.52	valid_loss:1.5500325 valid accuracy: 0.4679487
St

Steps:223
train_loss: 1.3901597 train accuracy: 0.48	valid_loss:1.369772 valid accuracy: 0.5653846
Steps:224
train_loss: 1.4280548 train accuracy: 0.49	valid_loss:1.4059062 valid accuracy: 0.52307695
Steps:225
train_loss: 1.4078046 train accuracy: 0.53	valid_loss:1.4148966 valid accuracy: 0.5205128
Steps:226
train_loss: 1.456597 train accuracy: 0.46	valid_loss:1.4011401 valid accuracy: 0.52692306
Steps:227
train_loss: 1.162219 train accuracy: 0.61	valid_loss:1.4212253 valid accuracy: 0.5346154
Steps:228
train_loss: 1.6057457 train accuracy: 0.49	valid_loss:1.3686484 valid accuracy: 0.54615384
Steps:229
train_loss: 1.1790334 train accuracy: 0.58	valid_loss:1.3586853 valid accuracy: 0.5474359
Steps:230
train_loss: 1.130701 train accuracy: 0.63	valid_loss:1.3733411 valid accuracy: 0.5551282
Steps:231
train_loss: 1.3292313 train accuracy: 0.55	valid_loss:1.3364357 valid accuracy: 0.5653846
Steps:232
train_loss: 1.3979585 train accuracy: 0.59	valid_loss:1.307155 valid accuracy: 0.5397436
St

Steps:306
train_loss: 1.268823 train accuracy: 0.59	valid_loss:1.4315861 valid accuracy: 0.51666665
Steps:307
train_loss: 1.310287 train accuracy: 0.52	valid_loss:1.340518 valid accuracy: 0.5423077
Steps:308
train_loss: 1.2289108 train accuracy: 0.59	valid_loss:1.2747692 valid accuracy: 0.55641025
Steps:309
train_loss: 1.4093666 train accuracy: 0.48	valid_loss:1.3006413 valid accuracy: 0.5474359
Steps:310
train_loss: 1.2025332 train accuracy: 0.55	valid_loss:1.3162296 valid accuracy: 0.5448718
Epoch: 11
Steps:311
train_loss: 1.3002089 train accuracy: 0.55	valid_loss:1.3487453 valid accuracy: 0.5679487
Steps:312
train_loss: 1.4238555 train accuracy: 0.51	valid_loss:1.3511602 valid accuracy: 0.5679487
Steps:313
train_loss: 1.8000629 train accuracy: 0.5	valid_loss:1.3809044 valid accuracy: 0.5628205
Steps:314
train_loss: 1.3193655 train accuracy: 0.58	valid_loss:1.3364594 valid accuracy: 0.5730769
Steps:315
train_loss: 1.2831926 train accuracy: 0.56	valid_loss:1.4071369 valid accuracy: 0.

...

Steps:15152
train_loss: 0.05685932 train accuracy: 0.99	valid_loss:3.0896056 valid accuracy: 0.8141026
Steps:15153
train_loss: 0.33105233 train accuracy: 0.94	valid_loss:3.0726976 valid accuracy: 0.8128205
Steps:15154
train_loss: 0.11449766 train accuracy: 0.98	valid_loss:3.0428174 valid accuracy: 0.8076923
Steps:15155
train_loss: 0.22479513 train accuracy: 0.97	valid_loss:3.0395558 valid accuracy: 0.81153846
Steps:15156
train_loss: 0.28814754 train accuracy: 0.97	valid_loss:3.0136075 valid accuracy: 0.8179487
Steps:15157
train_loss: 0.031046981 train accuracy: 0.98	valid_loss:3.0092986 valid accuracy: 0.82179487
Steps:15158
train_loss: 0.14780453 train accuracy: 0.95	valid_loss:2.9888737 valid accuracy: 0.82179487
Steps:15159
train_loss: 0.055306837 train accuracy: 0.99	valid_loss:2.9243574 valid accuracy: 0.82179487
Epoch: 490
Steps:15160
train_loss: 0.06894818 train accuracy: 0.97	valid_loss:2.8168933 valid accuracy: 0.824359
Steps:15161
train_loss: 0.12026535 train accuracy: 0.95	v

Steps:15232
train_loss: 0.040880907 train accuracy: 0.99	valid_loss:3.3172777 valid accuracy: 0.82179487
Steps:15233
train_loss: 0.2232847 train accuracy: 0.95	valid_loss:3.2469583 valid accuracy: 0.82564104
Steps:15234
train_loss: 0.48682618 train accuracy: 0.94	valid_loss:3.1004927 valid accuracy: 0.8346154
Steps:15235
train_loss: 0.20072442 train accuracy: 0.93	valid_loss:3.0634916 valid accuracy: 0.8346154
Steps:15236
train_loss: 0.16930346 train accuracy: 0.94	valid_loss:3.1570208 valid accuracy: 0.8192308
Steps:15237
train_loss: 0.18783964 train accuracy: 0.97	valid_loss:3.190975 valid accuracy: 0.8076923
Steps:15238
train_loss: 0.1803334 train accuracy: 0.95	valid_loss:3.228906 valid accuracy: 0.81153846
Steps:15239
train_loss: 0.09185841 train accuracy: 0.97	valid_loss:3.0746112 valid accuracy: 0.81153846
Steps:15240
train_loss: 0.07299256 train accuracy: 0.98	valid_loss:2.9410653 valid accuracy: 0.8153846
Steps:15241
train_loss: 0.3400222 train accuracy: 0.92	valid_loss:2.8285

Epoch: 495
Steps:15315
train_loss: 0.46347812 train accuracy: 0.89	valid_loss:3.3018646 valid accuracy: 0.8179487
Steps:15316
train_loss: 0.40648094 train accuracy: 0.94	valid_loss:3.1926532 valid accuracy: 0.82564104
Steps:15317
train_loss: 0.06087873 train accuracy: 0.97	valid_loss:2.965068 valid accuracy: 0.8333333
Steps:15318
train_loss: 0.073156245 train accuracy: 0.99	valid_loss:2.8029785 valid accuracy: 0.8333333
Steps:15319
train_loss: 0.11224787 train accuracy: 0.97	valid_loss:2.82331 valid accuracy: 0.824359
Steps:15320
train_loss: 0.41724303 train accuracy: 0.92	valid_loss:2.9305038 valid accuracy: 0.81666666
Steps:15321
train_loss: 0.28516918 train accuracy: 0.93	valid_loss:2.9850204 valid accuracy: 0.8051282
Steps:15322
train_loss: 0.2166605 train accuracy: 0.95	valid_loss:3.0253263 valid accuracy: 0.798718
Steps:15323
train_loss: 0.47747573 train accuracy: 0.92	valid_loss:2.9560158 valid accuracy: 0.7897436
Steps:15324
train_loss: 0.3160783 train accuracy: 0.94	valid_loss

Steps:15396
train_loss: 0.05364509 train accuracy: 0.98	valid_loss:2.9359884 valid accuracy: 0.824359
Steps:15397
train_loss: 0.22236206 train accuracy: 0.95	valid_loss:2.9198477 valid accuracy: 0.8269231
Steps:15398
train_loss: 0.054725006 train accuracy: 0.96	valid_loss:2.957153 valid accuracy: 0.824359
Steps:15399
train_loss: 0.19280237 train accuracy: 0.96	valid_loss:3.0333126 valid accuracy: 0.8269231
Steps:15400
train_loss: 0.234179 train accuracy: 0.94	valid_loss:3.0798519 valid accuracy: 0.82179487
Steps:15401
train_loss: 0.04805819 train accuracy: 0.99	valid_loss:3.1383083 valid accuracy: 0.8141026
Steps:15402
train_loss: 0.15360966 train accuracy: 0.99	valid_loss:3.1621234 valid accuracy: 0.81153846
Steps:15403
train_loss: 0.011162329 train accuracy: 1.0	valid_loss:3.2357748 valid accuracy: 0.8
Steps:15404
train_loss: 0.14714126 train accuracy: 0.95	valid_loss:3.2837877 valid accuracy: 0.79615384
Steps:15405
train_loss: 0.22138639 train accuracy: 0.98	valid_loss:3.2185621 val

Steps:15476
train_loss: 0.11674747 train accuracy: 0.98	valid_loss:3.0708795 valid accuracy: 0.81153846
Steps:15477
train_loss: 0.14170605 train accuracy: 0.96	valid_loss:2.9790366 valid accuracy: 0.80641025
Steps:15478
train_loss: 0.23835687 train accuracy: 0.92	valid_loss:3.0554907 valid accuracy: 0.8
Steps:15479
train_loss: 0.21801785 train accuracy: 0.93	valid_loss:3.0773137 valid accuracy: 0.8153846
Steps:15480
train_loss: 0.124569654 train accuracy: 0.96	valid_loss:3.1112504 valid accuracy: 0.8141026
Steps:15481
train_loss: 0.24767585 train accuracy: 0.97	valid_loss:3.0742369 valid accuracy: 0.8179487
Steps:15482
train_loss: 0.29981372 train accuracy: 0.95	valid_loss:2.998995 valid accuracy: 0.80641025
Steps:15483
train_loss: 0.14803354 train accuracy: 0.98	valid_loss:3.0288873 valid accuracy: 0.8076923
Steps:15484
train_loss: 0.19796407 train accuracy: 0.94	valid_loss:3.1627867 valid accuracy: 0.8025641
Steps:15485
train_loss: 0.31212294 train accuracy: 0.94	valid_loss:3.2494776